# Traducción de Script de Creación de la BBDD de T-SQL a Postgres

En el presente Notebook se ejecutarán todos los comandos necesarios para poder traducir el script de creación de la BBDD de "dialecto" T-SQL a Postgres. Para esto, se deben codificar los CSV con datos en un formato admisible para Postgres (UTF-8) y traducir el script en si. Para esto último, nos valdremos del uso de un LLM de Azure Open AI.

## Librerías



In [1]:
from dotenv import load_dotenv
from pathlib import Path
import csv
from openai import AzureOpenAI

## Variables globales



In [2]:
load_dotenv('../.env')

READ_DATA_PATH = '../data/database/tsql/db_creation'
FULL_SCRIPT_PATH = f'{READ_DATA_PATH}/instawdbdw.sql'
SQL_ENCODING = 'UTF-16'

SAMPLES_ENCODING = 'UTF16'
SAMPLES_DELIMITER = '|'

READ_DIALECT = 'tsql'
WRITE_DIALECT = 'postgres'

DB_NAME = 'adventure_works_dw'
SCHEMA = 'adventure_works'

TRANSPILING_MANUAL = 'MANUAL'
TRANSPILING_SQLGLOT = 'SQLGLOT'
TRANSPILING_LLM = 'LLM'

WRITE_DATA_PATH = '../data/database/postgres/db_creation'
TRANSPILED_SCRIPT_PATH = f'{WRITE_DATA_PATH}/create_adventure_works_postgres.sql'
OUTPUT_ENCODING = 'UTF-8'

## Recodificar ficheros CSV de UTF-16 a UTF-8

Procedemos a obtener todos los ficheros CSV almacenados en el directorio de lectura:

In [3]:
excluding_csv_list = [
    'DatabaseLog.csv',
    'sysdiagrams.csv',
]
csv_files_list = []

for file_path in Path(READ_DATA_PATH).rglob('*.csv'):
    exclude_csv = [file_path.as_posix().endswith(exclude_csv_name) for exclude_csv_name in excluding_csv_list]

    if any(exclude_csv):
        continue

    csv_files_list.append(file_path.as_posix())

csv_files_list

['../data/database/tsql/db_creation/DimAccount.csv',
 '../data/database/tsql/db_creation/DimCurrency.csv',
 '../data/database/tsql/db_creation/DimCustomer.csv',
 '../data/database/tsql/db_creation/DimDate.csv',
 '../data/database/tsql/db_creation/DimDepartmentGroup.csv',
 '../data/database/tsql/db_creation/DimEmployee.csv',
 '../data/database/tsql/db_creation/DimGeography.csv',
 '../data/database/tsql/db_creation/DimOrganization.csv',
 '../data/database/tsql/db_creation/DimProduct.csv',
 '../data/database/tsql/db_creation/DimProductCategory.csv',
 '../data/database/tsql/db_creation/DimProductSubcategory.csv',
 '../data/database/tsql/db_creation/DimPromotion.csv',
 '../data/database/tsql/db_creation/DimReseller.csv',
 '../data/database/tsql/db_creation/DimSalesReason.csv',
 '../data/database/tsql/db_creation/DimSalesTerritory.csv',
 '../data/database/tsql/db_creation/DimScenario.csv',
 '../data/database/tsql/db_creation/FactAdditionalInternationalProductDescription.csv',
 '../data/datab

Ahora, para cada fichero en el listado, procederemos a abrirlo con el encoding de lectura, y a guardarlo con el encoding de escritura:

In [4]:
SAMPLES_ABS_PATH = Path(WRITE_DATA_PATH).resolve().as_posix() + '/'
SAMPLES_ABS_PATH

'C:/python/ucm-tfm-grupo-4/data/database/postgres/db_creation/'

In [5]:
for csv_file_path in csv_files_list:
    csv_file_name = csv_file_path.split('/')[-1]
    csv_output_path = f'{SAMPLES_ABS_PATH}{csv_file_name}'

    with open(csv_file_path, 'r', encoding=SAMPLES_ENCODING) as r_f, \
        open(csv_output_path, 'w', encoding=OUTPUT_ENCODING, newline='') as w_f:

        reader = csv.reader(r_f, delimiter=SAMPLES_DELIMITER, quotechar='"', doublequote=True)
        writer = csv.writer(w_f, delimiter=SAMPLES_DELIMITER, quotechar='"', doublequote=True, quoting=csv.QUOTE_MINIMAL)

        for row in reader:
            writer.writerow(row)


## Traducción de Script T-SQL a Postgres

Cargamos todo el script:

In [6]:
with open(FULL_SCRIPT_PATH, 'r', encoding= SQL_ENCODING) as f:
    full_script = f.read()

Creamos funciones auxiliares y algunas definiciones adicionales a utilizar en el proceso:

In [7]:
REPLACEMENT_DICTS = [
    {'old': '$(DatabaseName)', 'new': DB_NAME},
    {'old': '$(SqlSamplesSourceDataPath)', 'new': SAMPLES_ABS_PATH},
    {'old': '[dbo]', 'new': f'[{SCHEMA}]'},
]


sections_params = [
    {
        'section': 'db',
        'start_marker': '-- ****************************************\n-- Create Database\n-- ****************************************',
        'end_marker': "*** Checking for $(DatabaseName) Database';\n",
    },
    {
        'section': 'tables',
        'start_marker': '-- ******************************************************\n-- Create tables\n-- ******************************************************',
        'end_marker': 'CREATE TABLE [dbo].[sysdiagrams](',
        'ignore_sub_section': [
            ('CREATE TABLE [dbo].[AdventureWorksDWBuildVersion](\n', 'CREATE TABLE [dbo].[DimAccount](\n')
        ],
    },
    {
        'section': 'load',
        'start_marker': '-- ******************************************************\n-- Load data\n-- ******************************************************',
        'end_marker': '-- ******************************************************\n-- Add Primary Keys\n-- ******************************************************',
        'ignore_sub_section': [
            ("PRINT 'Loading [dbo].[AdventureWorksDWBuildVersion]';\n", "PRINT 'Loading [dbo].[DimAccount]';\n"),
            ("PRINT 'Loading [dbo].[sysdiagrams]';", "-- ******************************************************\n-- Add Primary Keys\n-- ******************************************************")
        ],
    },
    {
        'section': 'primary_keys',
        'start_marker': '-- ******************************************************\n-- Add Primary Keys\n-- ******************************************************',
        'end_marker': '-- ******************************************************\n-- Add Indexes\n-- ******************************************************',
        'ignore_sub_section': [
            ("PRINT '*** Adding Primary Keys';\n", "ALTER TABLE [dbo].[DimAccount] WITH CHECK ADD\n")
        ],
    },
    {
        'section': 'indexes',
        'start_marker': '-- ******************************************************\n-- Add Indexes\n-- ******************************************************',
        'end_marker': '-- ****************************************\n-- Create Foreign key constraints\n-- ****************************************',
        'ignore_sub_section': [
            ('CREATE UNIQUE NONCLUSTERED INDEX [UK_principal_name] ON [dbo].[sysdiagrams]([principal_id],\t[name]) ON [PRIMARY];', '-- ****************************************\n-- Create Foreign key constraints\n-- ****************************************')
        ]
    },
    {
        'section': 'foreign_key',
        'start_marker': '-- ****************************************\n-- Create Foreign key constraints\n-- ****************************************',
        'end_marker': '-- ******************************************************\n-- Add database views.\n-- ******************************************************',
    }
]



def trf_strip_upper(line: str) -> str:
    """
    Removes leading/trailing whitespace and converts a string to uppercase.

    Args:
        line: The input string.

    Returns:
        The stripped and uppercased string.
    """

    return line.strip().upper()




def replace_values(script: str, replacement_dicts: list = REPLACEMENT_DICTS) -> str:
    """
    Replaces multiple values in a script string based on a list of dictionaries.

    Args:
        script: The input script string.
        replacement_dicts: A list of dictionaries, each with 'old' and 'new' keys
                           for the replacement. Defaults to REPLACEMENT_DICTS.

    Returns:
        The script string with replaced values.
    """

    for old_new_dict in replacement_dicts:
        script = script.replace(old_new_dict['old'], old_new_dict['new'])

    return script



def clean_lines(script: str, section: str = None) -> str:
    """
    Cleans up a script string by removing specific lines (PRINT, GO, comments)
    and applying section-specific cleaning rules.

    Args:
        script: The input script string.
        section: The name of the script section (e.g., 'indexes') to apply
                 section-specific rules. Defaults to None.

    Returns:
        The cleaned script string.
    """

    cleaned_lines = []

    for line in script.splitlines():
        if trf_strip_upper(line).startswith('PRINT'):
            continue

        if trf_strip_upper(line).startswith('-- '):
            continue

        cleaned_lines.append(line)


    return '\n'.join(cleaned_lines)



def llm_transpiling(script: str, input_dialect: str = READ_DIALECT, 
                    output_dialect:str = WRITE_DIALECT) -> str:
    """
    Uses an LLM to transpile specific sections of the SQL `script` from
    `input_dialect` to `output_dialect`.

    Args:
        script: The input script section string.
        input_dialect: The input SQL dialect.
        output_dialect: The output SQL dialect.

    Returns:
        The transpiled script section string.
    """

    system_prompt = f"""
    # CONTEXT #
    You are a database administrator and expert in SQL dialects. Your role is to **transpile DDL commands** from `{input_dialect}` to `{output_dialect}`. \
    The user will provide the contents of an `.sql` file containing `{input_dialect}` DDL commands.

    # OBJECTIVE #
    Your task is to output the **transpiled `{output_dialect}`-compatible DDL commands**, ready for execution.

    # INSTRUCTIONS #
    Your response must be a **plain text output** of valid DDL commands **for `{output_dialect}` only**, following **these strict rules**:

    - ❌ Do not use markdown tags (e.g., ` ```sql ` or similar).
    - ✅ Convert all entity names (tables, columns, schemas) from camelCase or PascalCase to `snake_case`.

    - ✅ Ensure the following database setup:
        - If the database `{DB_NAME}` already exists, drop it and recreate it from scratch. Then, connect to it.
        - Create schema `{SCHEMA}` within `{DB_NAME}`.
        - Drop schema `public` from `{DB_NAME}` if it exists.

    - ✅ Process table creation:
        - Create all tables under schema `{SCHEMA}`.
        - Do **not** define any `PRIMARY KEY`, `FOREIGN KEY`, or `INDEX` inside the `CREATE TABLE` statements.
        - Ignore `NOT NULL` constraints **only** for columns named `..._product_name` in the `dim_product` table.

    - ✅ Include data loading steps:
        - Use `COPY` from CSV for each table with this clause:  
            `WITH (DELIMITER '{SAMPLES_DELIMITER}', FORMAT CSV, HEADER FALSE, ENCODING '{OUTPUT_ENCODING.replace('-', '')}')`
        - ⚠️ **Do not modify the provided file path** for the CSV files under any circumstances.

    - ✅ After loading data with `COPY`, apply the following **in strict order**:
        1. Add `PRIMARY KEY` constraints using `ALTER TABLE` statements.
        2. Add `INDEXES` for relevant columns (e.g., those used in joins or foreign keys).
        3. Add `FOREIGN KEY` constraints using `ALTER TABLE ... ADD CONSTRAINT ... FOREIGN KEY ... REFERENCES ...`.

    - ⚠️ Ensure all constraint and index names are valid and do not exceed `{output_dialect}` maximum identifier length.
    - ✅ You may generate constraint and index names automatically using a consistent naming pattern based on table and column names.

    - ✅ If a data type, default value or constraint is not supported in `{output_dialect}`, choose the **closest compatible alternative** silently, without warnings or explanations.

    # RESPONSE FORMAT #
    Only output valid `{output_dialect}` DDL commands. Do not include comments or explanations unless they are part of the original SQL or required by the syntax.
    """

    client = AzureOpenAI()
    response = client.chat.completions.create(
        model= 'gpt-5-mini',
        # temperature= 0.0,
        messages= [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': script}
        ]
    )

    return response.choices[0].message.content


A continuación, implementamos el script que permitirá:

1.   Capturar las secciones de interés del script, excluyendo de ella los fragmentos no deseados.
2.   Aplicar transofrmaciones de limpieza sobre la estructura de las secciones de interés.
3.  Traducir del dialecto de entrada (T-SQL) al de salida (Postgres) las secciones deseadas.
4.  Finalmente, unificará todas secciones traducidas, y las exportará a un fichero `.sql`.




In [8]:
reduced_script = ''

for section_data in sections_params:
    section = section_data['section']
    print(f'Getting reduced section `{section}` ...')

    start_marker = section_data['start_marker']
    end_marker = section_data['end_marker']

    start_index = full_script.find(start_marker)
    end_index = full_script.find(end_marker)

    print(f'> Start index: {start_index}')
    print(f'> End index: {end_index}')

    if start_index != -1 and end_index != -1:
        section_script = full_script[start_index:end_index]
        section_data[f'{READ_DIALECT}_script'] = section_script
        print(f'> {READ_DIALECT} section obtained')


    cleaned_script = section_script
    if section_data.get('ignore_sub_section'):
        init_index = 0
        cleaned_script = ''

        for ignore_start, ignore_end in section_data['ignore_sub_section']:
            ignore_start_index = section_script.find(ignore_start)
            cleaned_script += section_script[init_index:ignore_start_index]

            init_index = section_script.find(ignore_end)

        cleaned_script += section_script[init_index:]
        print(f'> Sub-sections ignored')

    cleaned_script = replace_values(cleaned_script)
    cleaned_script = clean_lines(cleaned_script, section)
    section_data['cleaned_script'] = cleaned_script
    print(f'> {READ_DIALECT} section cleaned')

    reduced_script += section_data['start_marker']
    reduced_script += '\n' * 2
    reduced_script += section_data['cleaned_script']
    reduced_script += '\n' * 4

    print()

print('>> Reduced script obtained.')
print(">> LLM's transpiling ...")

with open(TRANSPILED_SCRIPT_PATH, 'w', encoding= OUTPUT_ENCODING) as f:
    f.write(llm_transpiling(reduced_script))

print()
print(f'--> Transpiled script saved in `{TRANSPILED_SCRIPT_PATH}` <--')

Getting reduced section `db` ...
> Start index: 2603
> End index: 2809
> tsql section obtained
> tsql section cleaned

Getting reduced section `tables` ...
> Start index: 7004
> End index: 21633
> tsql section obtained
> Sub-sections ignored
> tsql section cleaned

Getting reduced section `load` ...
> Start index: 22079
> End index: 31133
> tsql section obtained
> Sub-sections ignored
> tsql section cleaned

Getting reduced section `primary_keys` ...
> Start index: 31133
> End index: 36504
> tsql section obtained
> Sub-sections ignored
> tsql section cleaned

Getting reduced section `indexes` ...
> Start index: 36504
> End index: 40192
> tsql section obtained
> Sub-sections ignored
> tsql section cleaned

Getting reduced section `foreign_key` ...
> Start index: 40192
> End index: 47319
> tsql section obtained
> tsql section cleaned

>> Reduced script obtained.
>> LLM's transpiling ...

--> Transpiled script saved in `../data/database/postgres/db_creation/create_adventure_works_postgres